In [1]:
import pandas as pd
import re
import csv

In [2]:
# specify the file path to your csv. This list only contains two columns (Reg no. & PRN) from BM csv.
path = '/Users/deborahleem/scripts/SloaneRegNoList_tab.txt'

In [3]:
# load up your dataframe. separated by tab
df = pd.read_csv(path, sep = '\t')

In [4]:
# rename as we hate columns with whitespace
df = df.rename(columns={'Reg No.': 'Reg'})

In [5]:
# head gives you top 5 rows (it makes it easier to see)
df.head()

,Reg,PRN
0,NaN,CME6054
1,NaN,GAA62097
2,.129,MCN6596
3,.161,MCN7366
4,.167,MCN7054


In [6]:
len(df[df.Reg == 'SLO'])

12

In [7]:
# s.str.contains('^a', na=False)

In [8]:
df[df.Reg.str.contains('1693', na=False)]

,Reg,PRN
1765,SLMisc.1693,MCN298


In [9]:
df[df.Reg.duplicated()].sort_values('Reg')

,Reg,PRN
1042,Franks.88.+,RRC1768
1436,SLAntiq.282,YCA38945
1542,SLBCameos.73,MCN10727
1798,SLO,COC31438
1796,SLO,COC28898
1795,SLO,COC29213
1794,SLO,COC31515
1793,SLO,COC31436
1797,SLO,COC31846
1791,SLO,COC31873


In [10]:
#This script parse the latest Misc XML file to see if there are any matches
import xml.etree.ElementTree as ET
mytree = ET.parse('/Users/deborahleem/Documents/PhD/SloaneDBwork19/Miscellanies5Ah_MASTER_fixed_mrs.xml')

In [11]:
#extract digits from reg to regnum. if not digits, ignored.
df['Regnum'] = df['Reg']
for index, row in df.iterrows():
    num = str(row['Reg'])
    # If there's a ".", remove everything before it
    # (with multiple dots keeps going until it finds last one)
    # so "12.34.56" becomes "56"
    num = re.sub(r'.*\.', "", num)
    # Remove everything that's not a digit
    # So "AB123c" becomes "123"
    num = re.sub(r'\D', "", num)
    row['Regnum'] = num

In [12]:
df[df['Reg'] == '1756,0101.18']

,Reg,PRN,Regnum
86,"1756,0101.18",GAA35814,18


In [13]:
myroot = mytree.getroot()
print(myroot)
#print(myroot.tag)
#print(myroot[0].tag)


<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x11f24f638>


In [14]:
#every parent for every element
parent_map = {c:p for p in myroot.iter( ) for c in p}

In [15]:
#find div2 for every label inside it. 
#There are 10 div2 in total and they represent 7 different catalogues and 3 indices
div2_label_map = {}
for div2 in myroot.iter("{http://www.tei-c.org/ns/1.0}div2"):
    label = div2.iter('{http://www.tei-c.org/ns/1.0}label')
    label_text = ",".join([x.text or "None" for x in label])
    
    for catnum in div2.iter('{http://www.enlightenmentarchitectures.org}catnum'):
        div2_label_map[catnum] = label_text

In [16]:
#Using digits only from Reg No., result shows how many matches, matching Reg no. and div3 content

results = []

for catnum in myroot.iter('{http://www.enlightenmentarchitectures.org}catnum'):
    value = catnum.text
    if not value:
        continue
    value_num = re.sub(r'\D', "", value)
    if value_num == "":
        continue
    matching_rows = df[df['Regnum'] == value_num]
    if len(matching_rows) == 0:
        continue
    results.append((value, value_num, len(matching_rows), matching_rows, catnum))
    # do something with it 

# results.sort(key=lambda row: int(row[1]))


In [17]:
for (value, value_num, count, matching_rows, catnum) in results:
    parent = parent_map[catnum]
    parent_str = ET.tostring(parent, encoding='unicode')
    print(value, value_num, count)
    print(matching_rows)
    print(parent_str)
    print('----')
    

1. 1 22
                   Reg       PRN Regnum
97     1756,0101.190.1  GAA75617      1
314    1756,0101.969.1  GAA44897      1
467        1756,0104.1   GAA9141      1
471        1876,0301.1    CME884      1
503   1906,1128,0.32.1    RFC380      1
514      1913,0620,0.1   RFC2775      1
537      1928,0323,0.1    RFM553      1
580   1928,0323,0.46.1  RFM37594      1
706        1957,0118.1  GAA64617      1
708        1971,0422.1  GAA34605      1
710      1974,0617,0.1  RFM42709      1
756    1974,0617,0.2.1    RFM938      1
826        1976,0109.1  GAA15573      1
851        2009,5008.1  GAA85009      1
902   Am,SLMisc.1933.1    ENA226      1
999      As1891,0324.1   EAS5517      1
1214              SL.1  RRM49730      1
1233        SL.159.q.1  MCN13006      1
1236        SL.159.r.1   MCN5906      1
1239        SL.159.s.1    MCN761      1
1245    SLAIntaglios.1   MCN1553      1
1496       SLBCameos.1   MCN7733      1
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.

                    Reg       PRN Regnum
643   1928,0323,0.46.64  RFM37657     64
819    1974,0617,0.2.64   RFI2041     64
1289    SLAIntaglios.64   MCN6419     64
1841         SLRings.64  MCS13072     64
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-64.</ns1:catnum>
            <ns0:p>A Chinese <ns0:add rend="underline">compasse</ns0:add> sent me by <ns0:persName>M<ns0:hi rend="sup">r</ns0:hi>.-<ns0:lb /> Cunningham</ns0:persName> in a <ns0:placeName>Japan</ns0:placeName> box.</ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">199 C.</ns0:add>
              <ns0:add rend="red">25</ns0:add>
            </ns0:p>
          </ns0:div3>




          
----
-1879. 1879 1
              Reg      PRN Regnum
1776  SLMisc.1879  MCM6334   1879
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum 

                   Reg       PRN Regnum
534      1926,0324.131    GAA926    131
1335  SLAIntaglios.131   MCN5789    131
1684        SLMisc.131  MCN10524    131
1901       SLSeals.131  MCN11524    131
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-131.</ns1:catnum>
            <ns0:p>A <ns0:add rend="underline">powder horn</ns0:add> a drinking cup made <ns0:lb /> of the <ns0:measure type="sizeWeight">large</ns0:measure>
              <ns0:measure>part</ns0:measure> of a stagg or <ns0:lb />
              <ns0:add rend="underline">Elks</ns0:add>
              <ns0:material>horn?</ns0:material> Scripture histories carvd ---- } <ns0:note type="value">0. 10.
                0.</ns0:note>
              <ns0:add rend="pencil"> S. B</ns0:add></ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">✓216 C</ns0:add>
              <ns0:add rend="red">29. b.</ns0:add>
          

1799         SLO,M.196  CME309063    196
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">196.</ns1:catnum>
            <ns0:p>An artificiall <ns0:add rend="underline">eye</ns0:add> made by <ns0:persName>
                <ns0:add rend="underline">Verle.</ns0:add></ns0:persName></ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">199 a. 199 C</ns0:add>
              <ns0:add rend="red">27 b.</ns0:add>
            </ns0:p>
          </ns0:div3>

          
----
197. 197 3
                   Reg       PRN Regnum
100      1756,0101.197  GAA69215    197
1388  SLAIntaglios.197   MCN6172    197
1619     SLBCameos.197  MCN10701    197
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">197.</ns1:catnum>
            <ns0:p>A french <ns0:material>leather</ns0:material> 

              Reg       PRN Regnum
15           .265  MCN10385    265
1241       SL.265  RRM40883    265
1428  SLAntiq.265   MCN1130    265
1949  SLSeals.265   MCM4200    265
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">265.</ns1:catnum>
            <ns0:p>A <ns0:measure>litle</ns0:measure>
              <ns0:measure type="shape">ovall</ns0:measure>
              <ns0:material>serpentine</ns0:material>
              <ns0:add rend="underline">box</ns0:add>.</ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">244. d</ns0:add>
              <ns0:add rend="red">✓ 199 d</ns0:add>
            </ns0:p>
          </ns0:div3>

          
----
-267. 267 3
                Reg       PRN Regnum
16             .267  MCN10390    267
465   1756,0103.267  GAA32234    267
1659  SLBCameos.267   MCN7038    267
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www

...

1696        SLMisc.184   RRC3749    184
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-184.</ns1:catnum>
            <ns0:p>10 <ns0:add rend="underline">Cornelian</ns0:add>
              <ns0:measure type="sizeWeight">small</ns0:measure> knife hafts one<ns0:lb /> broken.</ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">187</ns0:add>
            </ns0:p>
          </ns0:div3>

          
----
-185. 185 3
                   Reg      PRN Regnum
1378  SLAIntaglios.185  MCN5493    185
1421       SLAntiq.185  MCN3154    185
1610     SLBCameos.185  MCN2589    185
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-185.</ns1:catnum>
            <ns0:p>A Scymiter haft of <ns0:note type="colour">red</ns0:note>
              <ns0:add rend="underline">Jasper</ns0:a

25. 25 10
                    Reg       PRN Regnum
89         1756,0101.25  GAA28712     25
496      1906,1128,0.25    RFC373     25
560      1928,0323,0.25   RFC2926     25
604   1928,0323,0.46.25  RFM37618     25
734    1974,0617,0.1.25    RFM917     25
780    1974,0617,0.2.25    RFM962     25
847        1978,0317.25  GAA66850     25
849        1978,0414.25  GAA67024     25
926   Am,SLMisc.1933.25  ENA22442     25
1517       SLBCameos.25   MCN7528     25
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">25.</ns1:catnum>
            <ns0:p>A <ns0:measure type="sizeWeight">small</ns0:measure>
              <ns0:note type="colour">whitish</ns0:note>
              <ns0:add rend="underline">agate box</ns0:add> &amp; cover -<ns0:lb /> round, with
                <ns0:measure>some</ns0:measure>
              <ns0:note type="colour">black</ns0:note> spotts in it <ns0:add rend="del">1 

1868         SLRings.99    MCN829     99
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-99.</ns1:catnum>
            <ns0:p><ns0:add rend="underline">Another</ns0:add> sett in fueille <ns0:note type="colour">green</ns0:note>
              <ns0:note type="colour">gold</ns0:note>
              <ns0:note type="value">3. 4. 6.</ns0:note><ns0:lb /> w<ns0:hi rend="sup">t</ns0:hi>.
                <ns0:measure>some</ns0:measure>
              <ns0:measure type="sizeWeight">small</ns0:measure> rubies &amp; enamel'd <ns0:note type="value">3. 4. 6.</ns0:note></ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">193</ns0:add>
              <ns0:metamark rend="pencil">X</ns0:metamark>
            </ns0:p>
          </ns0:div3>
          
----
-100. 100 3
                     Reg       PRN Regnum
679   1928,0323,0.46.100  RFM37693    100
1314    SLAIntaglios.100   MCN5117    

1601     SLBCameos.174   MCN441    174
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-174.</ns1:catnum>
            <ns0:p>The <ns0:add rend="underline">same</ns0:add> w<ns0:hi rend="sup">t</ns0:hi>. 2 <ns0:note type="colour">red</ns0:note> Spotts on each<ns0:lb /> end a seal.</ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">✓ 184</ns0:add>
            </ns0:p>
          </ns0:div3>

          
----
-175. 175 3
                   Reg       PRN Regnum
852         Af,N/N.175  EAF28964    175
1368  SLAIntaglios.175   MCN5971    175
1602     SLBCameos.175  MCN12270    175
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-175.</ns1:catnum>
            <ns0:p>A <ns0:measure type="sizeWeight">small</ns0:measure>
              <ns0:metamark function="carets">

1657  SLBCameos.249  MCN11585    249
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-249.</ns1:catnum>
            <ns0:p>A <ns0:measure type="sizeWeight">small</ns0:measure>
              <ns0:add rend="underline">Cornelian cup</ns0:add>. oval.</ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">192</ns0:add>
            </ns0:p>
          </ns0:div3>

          
----
-250. 250 1
               Reg       PRN Regnum
456  1756,0103.250  GAA31357    250
<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">
            <ns1:catnum place="margin" type="primary">-250.</ns1:catnum>
            <ns0:p><ns0:add rend="underline">Another</ns0:add>. ovall</ns0:p>
            <ns0:p>
              <ns0:add rend="pencil">192</ns0:add>
            </ns0:p>
          </ns0:div3>

          
----
-251. 251 1
              Reg   

In [18]:
# Matching and inserting xml text to my Excel starts here

In [19]:
# now we get values where match = 1 
resx = [x for x in results if x[2] == 1]

In [20]:
res = {}
for val in resx:
    # get the dataframe    
    df = val[3]
#     print(df.Reg)
    reg = df.loc[:,'Reg']
#     print(val.values[0])
    if 'SLMisc' in reg.values[0]:
        name = reg.values[0]
        p = parent_map[val[4]]
        s = ET.tostring(p, encoding='unicode')
        res[name] = s
# print(res)
from pprint import pprint as pp

In [21]:
# This saves to excel the vale.
# steps are:
# 1: go through all the values in our dict (res). 
# 2. Go through all the values in our excel.
# 3. if value in step 1 == step2 value then:
# 4 target the cell adjacent (Cell C) with the value of the dictionary (res)
from openpyxl import Workbook, load_workbook
ref = load_workbook(filename = '/Users/deborahleem/sloane_db_list_modified.xlsx')
# vals exist for name inside colB
sheet_ranges = ref['objects']
for k,v in res.items():
    for i in range(1,1953):
        pos = 'B' + str(i)
    #     print(val)
        val = (sheet_ranges[pos].value)
#         if 'SLMisc' in val:
#         print(val)
        if val == k:
            target = 'C' + str(i)
            sheet_ranges[target] = v
            print('found ' + k + ' here: ' +  pos)
ref.save(filename='/Users/deborahleem/sloane_db_list_modified.xlsx')

found Am,SLMisc.1842 here: B902
found SLMisc.1866 here: B1777
found SLMisc.1879 here: B1778
found Am,SLMisc.1933 here: B903
found Am,SLMisc.1934 here: B945
found Af,SLMisc.1935 here: B864
found As,SLMisc.145 here: B954
found SLMisc.1984 here: B1779
found SLMisc.1985 here: B1780
found SLMisc.1987 here: B1781
found SLMisc.2010 here: B1783
found Am,SLMisc.2089 here: B949
found SLMisc.2091 here: B1786
found SLMisc.301 here: B1712
found SLMisc.2101 here: B1787
found SLMisc.2102 here: B1788
found Af,SLMisc.2105 here: B866
found SLMisc.342 here: B1714
found Eu,SLMisc.401 here: B1037
found SLMisc.418 here: B1716
found Af,SLMisc.424 here: B858
found SLMisc.562 here: B1717
found Am,SLMisc.572 here: B878
found As,SLMisc.664 here: B963
found Eu,SLMisc.755 here: B1038
found Am,SLMisc.758 here: B882
found SLMisc.795 here: B1719
found As,SLMisc.884 here: B964
found Am,SLMisc.885 here: B883
found Am,SLMisc.887 here: B885
found As,SLMisc.922 here: B965
found Am,SLMisc.981 here: B886
found SLMisc.1033 h

In [22]:
parent = parent_map[resx[0][4]]
ET.tostring(parent, encoding='unicode')

'<ns0:div3 xmlns:ns0="http://www.tei-c.org/ns/1.0" xmlns:ns1="http://www.enlightenmentarchitectures.org">\n            <ns1:catnum place="margin" type="primary">-1842.</ns1:catnum>\n            <ns0:p>A <ns0:measure>piece</ns0:measure> of <ns0:add rend="underline"><ns0:material>wood</ns0:material></ns0:add>\n              to cover the <ns0:add rend="underline">eyes</ns0:add> &amp; <ns0:measure>part</ns0:measure> of\n              the<ns0:lb /> nose to be fastened to the head by <ns0:material>beaver</ns0:material> straps<ns0:lb />\n                w<ns0:hi rend="sup">t</ns0:hi>. 2 chinks in the wood to see through. <ns0:add rend="underline">Id</ns0:add></ns0:p>\n            <ns0:p>\n              <ns0:add rend="pencil">&lt; 226</ns0:add>\n              <ns0:add rend="del">244</ns0:add>\n              <ns0:add rend="red">28. C.</ns0:add>\n            </ns0:p>\n          </ns0:div3>\n\n          '

In [23]:
with open('result_modified.csv', 'w') as file:
    writer = csv.writer(file)
    for (value, value_num, count, matching_rows, catnum) in results:
        parent = parent_map[catnum]
        parent_str = ET.tostring(parent, encoding='unicode')
        div2_str = div2_label_map[catnum]
        writer.writerow([value, value_num, count, div2_str])
        for i, row in matching_rows.iterrows():
            writer.writerow(row)
        writer.writerow([parent_str])
        writer.writerow(["----"])
        

In [24]:
results

[('1.', '1', 22,                    Reg       PRN Regnum
  97     1756,0101.190.1  GAA75617      1
  314    1756,0101.969.1  GAA44897      1
  467        1756,0104.1   GAA9141      1
  471        1876,0301.1    CME884      1
  503   1906,1128,0.32.1    RFC380      1
  514      1913,0620,0.1   RFC2775      1
  537      1928,0323,0.1    RFM553      1
  580   1928,0323,0.46.1  RFM37594      1
  706        1957,0118.1  GAA64617      1
  708        1971,0422.1  GAA34605      1
  710      1974,0617,0.1  RFM42709      1
  756    1974,0617,0.2.1    RFM938      1
  826        1976,0109.1  GAA15573      1
  851        2009,5008.1  GAA85009      1
  902   Am,SLMisc.1933.1    ENA226      1
  999      As1891,0324.1   EAS5517      1
  1214              SL.1  RRM49730      1
  1233        SL.159.q.1  MCN13006      1
  1236        SL.159.r.1   MCN5906      1
  1239        SL.159.s.1    MCN761      1
  1245    SLAIntaglios.1   MCN1553      1
  1496       SLBCameos.1   MCN7733      1, <Element '{http://

In [25]:
for catnum in myroot.iter('{http://www.enlightenmentarchitectures.org}catnum'):
    print(catnum)

<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c0048>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c0408>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c0638>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c0a48>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c0c78>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c0f48>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c71d8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c7458>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c7908>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8c7f48>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8ce368>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8ce9a8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x11f8cedb8>

<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203e24f8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203e2908>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203e2c78>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203e2f98>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203e84a8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203e8728>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203e8bd8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203e8f48>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203ee458>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203ee908>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203eed68>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203f3278>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x1203f35e8>

<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b59a48>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b5e098>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b5e548>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b5eb88>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b63098>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b635e8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b63ef8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b681d8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b68458>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b68688>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b68a48>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b68ef8>
<Element '{http://www.enlightenmentarchitectures.org}catnum' at 0x120b6e688>

In [26]:
#Extracting persName from Misc XML along with catnum and div2 label
people_csv_data = []

for div3 in myroot.iter("{http://www.tei-c.org/ns/1.0}div3"):
    persons = list(div3.iter('{http://www.tei-c.org/ns/1.0}persName'))
    catnums = list(div3.iter('{http://www.enlightenmentarchitectures.org}catnum'))
    if len(persons) > 0:
        people_csv_data.append(["div3"])
        print("div3")
#         print(len(persons))
#         print(len(catnums))
# itertext gets all text inside element, then we join them
        for person in persons:
#             print(ET.tostring(person, encoding='unicode'))
            name = re.sub(r'\s+', " ", ''.join(person.itertext()))
            print(name)
            people_csv_data.append(["person", name])

        for catnum in catnums:
#             print(ET.tostring(catnum, encoding='unicode'))
            name = ''.join(catnum.itertext())
            print(name)
            print(div2_label_map[catnum])
            people_csv_data.append(["catnum", name])
            people_csv_data.append(["div2", div2_label_map[catnum]])
        print("")
    

div3
Mr. Amyand.
-1799.
Miscellanies.

div3
Mr. Theobalds.
-1801.
Miscellanies.

div3
Mr. Roberts
-1806.
Miscellanies.

div3
Mr. M
-1807.
Miscellanies.

div3
 Dr. Stuart
-1809.
Miscellanies.

div3
Mr. Mc. Cormick
1812.
Miscellanies.

div3
Mr. Hodsun
-1813.
Miscellanies.

div3
Mr. Launce
-1816.
Miscellanies.

div3
 father Fontenay
___7.
Miscellanies.

div3
Mr. Cunningham
___9.
Miscellanies.

div3
Dr. Waldo
12.
Miscellanies.

div3
Capt. Walker.
-1820.
Miscellanies.

div3
Capt. Walker
-1822
Miscellanies.

div3
IsaacWaldron Esq r
-1831.
Miscellanies.

div3
Abraham Waldron?
-1835.
Miscellanies.

div3
 Mr. Courtens
20.
Miscellanies.

div3
 Mr. Edwards
-1838.
Miscellanies.

div3
Dr. Massy
-1843.
Miscellanies.

div3
 Mr. Edwards
-1844.
Miscellanies.

div3
 M r. Collinson.
-1845.
Miscellanies.

div3
 Monsr. Gosset
-1847.
Miscellanies.

div3
 Dr. Mortimer
-1851.
Miscellanies.

div3
 Mr. Gordon
-1852.
Miscellanies.

div3
D r. Massy
-1856.
Miscellanies.

div3
Dr. Douce.
-1857.
Miscellanies.

div3


Miscellanies.

div3
Mr. Skinner
 Mr. Lockyer.
-1281.
Miscellanies.

div3
D r. Hubbert
-1284.
Miscellanies.

div3
Dr. Hubert
-1307.
Miscellanies.

div3
M r. Elking
-1332.
Miscellanies.

div3
Mr. Pitts.
 Dr. Mortimer
-1336.
Miscellanies.

div3
Mr. Harris
-1343.
Miscellanies.

div3
Dr. Burnett
-1344.
Miscellanies.

div3
 Mr. Gale
-1346.
Miscellanies.

div3
 Dr. Hoffman
Dr. Plumptree
-1347.
Miscellanies.

div3
Mr. John Heathcote
-1349.
Miscellanies.

div3
Mrs. Shepherd
-1353.
Miscellanies.

div3
Mr. Clerk
-1368.
Miscellanies.

div3
Mrs. Dering
-1371.
Miscellanies.

div3
Mr. Clerk
-1373.
Miscellanies.

div3
governor Bennet
-1382.
Miscellanies.

div3
 Mr. Bell
-1383.
Miscellanies.

div3
Mr. Elking
-1385.
Miscellanies.

div3
Mr. Clerk
-1387.
Miscellanies.

div3
Dr. Varin
-1389.
Miscellanies.

div3
Mr. Fabre
-1395.
Miscellanies.

div3
Mr. P B admore
-1396.
Miscellanies.

div3
 Pet
-1397.
Miscellanies.

div3
Mr. Wilsons
-1398.
Miscellanies.

div3
Mr. Lyell
-1422.
Miscellanies.

div3
 Dr. Woodwa

Pictures &c.

div3
Mr. Cooke
-74.
Pictures &c.

div3
Mr. Cooke
-77.
Pictures &c.

div3
Mareschall
-78.
Pictures &c.

div3
Tholouze 
82.
Pictures &c.

div3
 Dr. Harvy
-90.
Pictures &c.

div3
Mrs. Fuller
 Taverner
-92.
Pictures &c.

div3
Mr. Sam
-93.
Pictures &c.

div3
Wissen.
-97.
Pictures &c.

div3
Rossi's
-98.
Pictures &c.

div3
 Mrs. Courten
 Vandike
103.
Pictures &c.

div3
Lord Exeter
104.
Pictures &c.

div3
 Silvestre
108.
Pictures &c.

div3
 Merian
-109.
Pictures &c.

div3
 Dr. Huntington
-120.
Pictures &c.

div3
Mr. Langley
-129.
Pictures &c.

div3
 Mr. Courten
130.
Pictures &c.

div3
Mrs. AnneRose
 Taverner
-132.
Pictures &c.

div3
 Holben
133.
Pictures &c.

div3
Mr. Cooke
135.
Pictures &c.

div3
Miller
-137.
Pictures &c.

div3
Mr. Cook
-138.
Pictures &c.

div3
 Vesalius
 Antony More
147.
Pictures &c.

div3
Young Fran: Mercur. Vanhelmont
 Mr. Murray
148.
Pictures &c.

div3
 Mr. Courten
151.
Pictures &c.

div3
 Mumper
152.
Pictures &c.

div3
Mr. Cook
-153.
Pictures &c.

div3
 Sr.

 Mr. Dupuys
-215.
Agate cups botles spoons &c.

div3
 Mr. Bell
-220.
Agate cups botles spoons &c.

div3
 Mr. Ranby
-227.
Agate cups botles spoons &c.

div3
Mrs. Shepherd
-229.
Agate cups botles spoons &c.

div3
madam Montmorency
-302.
Agate cups botles spoons &c.



In [27]:
people_csv_data

[['div3'],
 ['person', 'Mr. Amyand.'],
 ['catnum', '-1799.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', 'Mr. Theobalds.'],
 ['catnum', '-1801.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', 'Mr. Roberts'],
 ['catnum', '-1806.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', 'Mr. M'],
 ['catnum', '-1807.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', ' Dr. Stuart'],
 ['catnum', '-1809.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', 'Mr. Mc. Cormick'],
 ['catnum', '1812.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', 'Mr. Hodsun'],
 ['catnum', '-1813.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', 'Mr. Launce'],
 ['catnum', '-1816.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', ' father Fontenay'],
 ['catnum', '___7.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', 'Mr. Cunningham'],
 ['catnum', '___9.'],
 ['div2', 'Miscellanies.'],
 ['div3'],
 ['person', 'Dr. Waldo'],
 ['catnum', '12.'],
 ['div2', 'Miscellanies.'],
 ['div3'],


In [28]:
with open('people.csv', 'w') as file:
    writer = csv.writer(file)
    for row in people_csv_data:
        writer.writerow(row)
        

In [29]:
#Extracting placeName from Misc XML along with catnum and div2 label
places_csv_data = []

for div3 in myroot.iter("{http://www.tei-c.org/ns/1.0}div3"):
    places = list(div3.iter('{http://www.tei-c.org/ns/1.0}placeName'))
    catnums = list(div3.iter('{http://www.enlightenmentarchitectures.org}catnum'))
    if len(places) > 0:
        places_csv_data.append(["div3"])
        print("div3")
        for place in places:
            # replace any number of consecutive spaces/newlines with one space
            name = re.sub(r'\s+', " ", ''.join(place.itertext()))
            print(name)
            places_csv_data.append(["place", name])
        for catnum in catnums:
            name = ''.join(catnum.itertext())
            print(name)
            print(div2_label_map[catnum])
            places_csv_data.append(["catnum", name])
            places_csv_data.append(["div2", div2_label_map[catnum]])
        print("")
    

div3
East Indies
-1799.
Miscellanies.

div3
Norway
-1801.
Miscellanies.

div3
Lisbon
-1807.
Miscellanies.

div3
Lisbone
-1808.
Miscellanies.

div3
Malacca
-1809.
Miscellanies.

div3
China
1810.
Miscellanies.

div3
Lisbon
Brasile
1812.
Miscellanies.

div3
China
-1813.
Miscellanies.

div3
Guinea
-1816.
Miscellanies.

div3
Malabar
-1818.
Miscellanies.

div3
England
-1819.
Miscellanies.

div3
London
1.
Miscellanies.

div3
Coromandel
4.
Miscellanies.

div3
China
___7.
Miscellanies.

div3
Persian
8.
Miscellanies.

div3
Tunquin
___9.
Miscellanies.

div3
China
Suratte
12.
Miscellanies.

div3
New England
-1820.
Miscellanies.

div3
Berbices
Surinam
West Indies
-1822
Miscellanies.

div3
Surinam
-1823.
Miscellanies.

div3
Guinea
-1830.
Miscellanies.

div3
hope near Bristol
Bristol
-1831.
Miscellanies.

div3
Guinea
-1833.
Miscellanies.

div3
China
13.
Miscellanies.

div3
Isles of Scotland
14.
Miscellanies.

div3
Turkey
15.
Miscellanies.

div3
Brasile
Brasile
__16.
Miscellanies.

div3
Brasile
21.
Mi

North America
-1534.
Miscellanies.

div3
America
-1535.
Miscellanies.

div3
Ireland
-1537.
Miscellanies.

div3
Turkish
Constantinople
Smyrna
-1538.
Miscellanies.

div3
Turkish
Smyrna
-1539.
Miscellanies.

div3
Turkish
Constantinople
Smyrna
-1540.
Miscellanies.

div3
China
-1542.
Miscellanies.

div3
France
-1543.
Miscellanies.

div3
Bohemia
-1544.
Miscellanies.

div3
Polonia
Germany
-1546.
Miscellanies.

div3
Windsor
-1547.
Miscellanies.

div3
Paris 
-1550
Miscellanies.

div3
Westminster
-1554.
Miscellanies.

div3
Egypt
-1556.
Miscellanies.

div3
Japan
-1563.
Miscellanies.

div3
Japan
-1565.
Miscellanies.

div3
Nuremberg
-1566.
Miscellanies.

div3
Dieppe
-1577.
Miscellanies.

div3
East Indies
-1588.
Miscellanies.

div3
Sweden
Norway
-1589.
Miscellanies.

div3
America
-1590.
Miscellanies.

div3
Windsor
-1591.
Miscellanies.

div3
Japan
-1592.
Miscellanies.

div3
Sheffield
-1595.
Miscellanies.

div3
Sheffield
1596.
Miscellanies.

div3
Holland
-1602.
Miscellanies.

div3
Turkish
Berlin
China

In [30]:
with open('places.csv', 'w') as file:
    writer = csv.writer(file)
    for row in places_csv_data:
        writer.writerow(row)
        